### Importing the needed libraries

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

### Getting the page into xml format

In [2]:
url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(url,'lxml')

### Finding the table tag, class tag and its value to get the desired table

In [3]:
table=soup.find('table',{'class':'wikitable sortable'})

   ### Creating a list of headers. The headers are marked with the 'th' tag

In [4]:
ths=table.find_all('th')
headings = [th.text.strip() for th in ths]


### Creating lists data for each header from the rows, the rows are marked with the 'tr' tag, and the data with 'td' tag

In [5]:
Postcode=[]
Borough=[]
Neighbourhood=[]
for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        Postcode_, Borough_, Neighbourhood_=[td.text.strip() for td in tds[:3]]
        Postcode.append(Postcode_)
        Borough.append(Borough_)
        Neighbourhood.append(Neighbourhood_)

### Creating the data frame

In [6]:
df1=pd.DataFrame(columns=headings)
df1['Postcode']=Postcode
df1['Neighbourhood']=Neighbourhood
df1['Borough']=Borough




### Removing the rows where the Borough is not assigned

In [11]:
df1=df1[df1.Borough!='Not assigned']

### Replacing the Neighbourhoods that are not assigned with the Borough

In [12]:
df1['Neighbourhood']=df1['Neighbourhood'].replace('Not assigned', df1['Borough'])

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


### Combining the Neighbourhoods for the same Postcode and Borough

In [13]:
grpby=df1.groupby(['Postcode','Borough'], as_index=False)
df=grpby.agg(lambda x: ', '.join(x))
df.head(20)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### Shape of the DataFrame

In [14]:
df.shape

(103, 3)

### Adding Latitude and Longitude

In [24]:
df_geo=pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [25]:
df1=pd.concat([df, df_geo], join='inner', axis=1)
df1.drop('Postal Code', inplace=True, axis=1)

In [26]:
df1.head(25)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.763573,-79.188711
3,M4A,North York,Victoria Village,43.770992,-79.216917
4,M5A,Downtown Toronto,Harbourfront,43.773136,-79.239476
5,M5A,Downtown Toronto,Regent Park,43.744734,-79.239476
6,M6A,North York,Lawrence Heights,43.727929,-79.262029
7,M6A,North York,Lawrence Manor,43.711112,-79.284577
8,M7A,Queen's Park,Queen's Park,43.716316,-79.239476
10,M9A,Etobicoke,Islington Avenue,43.757410,-79.273304
11,M1B,Scarborough,Rouge,43.750072,-79.295849
12,M1B,Scarborough,Malvern,43.794200,-79.262029


### Importing needed libraries for map clustering

In [18]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


### Define dataset for Toronto

In [97]:
#I've changed my dataframe on top so as to get the dataframe not aggregated by Neighborhoods

df1=df1[df1.Borough.apply(lambda x: 'Toronto' in x)]
df1.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)
df=df1

In [98]:
df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels
4,M5A,Downtown Toronto,Harbourfront,43.773136,-79.239476,0
5,M5A,Downtown Toronto,Regent Park,43.744734,-79.239476,3
17,M5B,Downtown Toronto,Ryerson,43.803762,-79.363452,0
18,M5B,Downtown Toronto,Garden District,43.778517,-79.346556,0
34,M5C,Downtown Toronto,St. James Town,43.725882,-79.315572,0


### Define Foursquare Credentials and Version

In [99]:
CLIENT_ID = 'BKFOSIM2OYDU4YH4VTEKP4OBR5M2OC1JET20BCZJGUQ5RM02' # your Foursquare ID
CLIENT_SECRET = 'VDZXDEXA44TNI5H4WB5KNJEUEFOIAOTVVPJGEHH5Z3ZOGQND' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: BKFOSIM2OYDU4YH4VTEKP4OBR5M2OC1JET20BCZJGUQ5RM02
CLIENT_SECRET:VDZXDEXA44TNI5H4WB5KNJEUEFOIAOTVVPJGEHH5Z3ZOGQND


### Explore the neighborhoods of Toronto

In [100]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit=100'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Creating Toronto venues DataFrame with the function

In [113]:
toronto_venues=getNearbyVenues(df['Neighborhood'], df['Latitude'], df['Longitude'])
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.773136,-79.239476,Federick Restaurant,43.774697,-79.241142,Hakka Restaurant
1,Harbourfront,43.773136,-79.239476,Drupati's Roti & Doubles,43.775222,-79.241678,Caribbean Restaurant
2,Harbourfront,43.773136,-79.239476,Thai One On,43.774468,-79.241268,Thai Restaurant
3,Harbourfront,43.773136,-79.239476,Centennial Recreation Centre,43.774593,-79.236500,Athletics & Sports
4,Harbourfront,43.773136,-79.239476,TD Canada Trust,43.774952,-79.241343,Bank


### Check the size of the resulting dataframe

In [114]:
toronto_venues.shape

(674, 7)

### Check how many venues were returned for each neighborhood

In [115]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,13,13,13,13,13,13
Berczy Park,1,1,1,1,1,1
Brockton,11,11,11,11,11,11
Central Bay Street,80,80,80,80,80,80
Christie,100,100,100,100,100,100
Design Exchange,6,6,6,6,6,6
Dovercourt Village,16,16,16,16,16,16
Dufferin,5,5,5,5,5,5
Exhibition Place,2,2,2,2,2,2


## Analyze Each Neighborhood

In [116]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hobby Shop,Hockey Arena,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Luggage Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Park,Persian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supplement Shop,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [117]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hobby Shop,Hockey Arena,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Luggage Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Park,Persian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supplement Shop,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Adelaide,0.0000,0.0,0.076923,0.076923,0.076923,0.153846,0.153846,0.153846,0.0000,0.0,0.00,0.0000,0.0,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.0,0.076923,0.00,0.076923,0.00,0.00,0.0,0.0000,0.0000,0.00,0.0000,0.0,0.0,0.0,0.000,0.0,0.0000,0.00,0.000000,0.0,0.0,0.0000,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.00,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0000,0.000000,0.0000,0.0,0.0,0.0000,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.000000,0.00,0.0000,0.0,0.076923,0.0,0.0,0.0,0.00,0.0,0.0000,0.000,0.0,0.0,0.00,0.000,0.00,0.0,0.0000,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0000,0.0000,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0000,0.0,0.0000,0.0,0.000000,0.000000,0.0,0.00,0.0000,0.0,0.0000,0.00,0.0,0.0000,0.00,0.0,0.0,0.00,0.000,0.00,0.000000,0.076923,0.0000,0.0,0.00,0.0,0.00,0.0000,0.0,0.000,0.00,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0000,0.000,0.00,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0000,0.0,0.00
1,Berczy Park,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.00,0.0000,0.0,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.00,0.00,0.0,0.0000,0.0000,0.00,0.0000,0.0,0.0,0.0,0.000,0.0,0.0000,0.00,0.000000,0.0,0.0,0.0000,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.00,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0000,0.000000,0.0000,0.0,0.0,0.0000,0.0,0.00,0.0,

### Define a function for top venus for a neighborhood

In [118]:
num_top_venues=10
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create a dataframe to show top10 venues for each neighborhood

In [119]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Airport Lounge,Airport Terminal,Airport Service,Sculpture Garden,Boat or Ferry,Harbor / Marina,Boutique,Airport Gate,Airport Food Court,Airport
1,Berczy Park,Restaurant,Women's Store,Electronics Store,Drugstore,Donut Shop,Dog Run,Discount Store,Diner,Dessert Shop,Department Store
2,Brockton,Grocery Store,Pizza Place,Coffee Shop,Sandwich Place,Video Store,Pharmacy,Fried Chicken Joint,Fast Food Restaurant,Beer Store,Cuban Restaurant
3,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Ice Cream Shop,Café,Bubble Tea Shop,Burger Joint,Bar,Falafel Restaurant,Salad Place
4,Christie,Coffee Shop,Café,Steakhouse,Thai Restaurant,American Restaurant,Gym,Clothing Store,Restaurant,Hotel,Bar


## Cluster Neighborhoods with K-Means

In [120]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

#### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [121]:
toronto_merged = df

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M5A,Downtown Toronto,Harbourfront,43.773136,-79.239476,0,Lounge,Bank,Hakka Restaurant,Bakery,Caribbean Restaurant,Thai Restaurant,Fried Chicken Joint,Athletics & Sports,Women's Store,Dessert Shop
5,M5A,Downtown Toronto,Regent Park,43.744734,-79.239476,3,Playground,Jewelry Store,Women's Store,Cuban Restaurant,Drugstore,Donut Shop,Dog Run,Discount Store,Diner,Dessert Shop
17,M5B,Downtown Toronto,Ryerson,43.803762,-79.363452,0,Golf Course,Mediterranean Restaurant,Dog Run,Fast Food Restaurant,Pool,Cuban Restaurant,Donut Shop,Discount Store,Diner,Dessert Shop
18,M5B,Downtown Toronto,Garden District,43.778517,-79.346556,0,Clothing Store,Fast Food Restaurant,Coffee Shop,Toy / Game Store,Shoe Store,Electronics Store,Bakery,Metro Station,Liquor Store,Luggage Store
34,M5C,Downtown Toronto,St. James Town,43.725882,-79.315572,0,Coffee Shop,Pizza Place,Portuguese Restaurant,Hockey Arena,Intersection,Cuban Restaurant,Donut Shop,Dog Run,Discount Store,Diner


## Visualize the Clusters

In [112]:
# create map
latitude=43.6532
longitude=-79.3832
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Observations:
As we can see on the Map, we have many red points that means that a cluster is dominant above the others, and each other cluster has just one observations. That can mean that the neighborhoods of Toronto are similar to each other.